#### 1. 卷积样例

In [2]:
import tensorflow as tf

In [3]:
# input输入图像是一个四维矩阵
# 第1维是第X个图像
# 第2第3维是长度宽度为32*32
# 第4维是RGB颜色通道数为3的矩阵

input = tf.placeholder(
    name='input',
    shape=[None, 32, 32, 3],
    dtype=tf.float32)

    

# filter是一个四维矩阵
# 第1第2维是过滤器尺寸width和height（5*5），
# 第3维当前层深度（3）
# 第4维是过滤器深度（16）
# 初始化为正态分布的随机变量
filter_weight = tf.get_variable(
    name='weights', 
    shape=[5, 5, 3, 16], 
    initializer=tf.truncated_normal_initializer(stddev=0.1)
)

# biases偏置量是一个一维矩阵
# 第1维是偏置量的深度（16）
# 初始化为数值都是0.1的一维矩阵
biases = tf.get_variable(
    name='biases', 
    shape=[16], 
    initializer=tf.constant_initializer(0.1)
)

# tf.nn.conv2d的strides步长参数
# strides步长是一个四维矩阵
# 第1维和第4维必须为1，因为卷积层的步长只对矩阵的长宽有效
# 第2维和第3维是长和宽的步长
# 
# tf.nn.conv2d的padding参数只提供两种SAME以及VAILD
# SAME表示添加0填充
# VAILD表示不添加填充
conv = tf.nn.conv2d(
    input=input, 
    filter=filter_weight, 
    strides=[1,1,1,1], 
    padding='SAME'
)

# tf.nn.bias_add提供了一个方便的函数给节点添加偏置量（注意这里不能直接使用加法，因为矩阵上不同位置上的节点都需要加上同样的偏置量）
# 例如下一层神经网络的大小为2*2，但是偏置量只有一个数（因为深度为1），而2*2矩阵的每一个项都需要加上这个偏置量。
bias = tf.nn.bias_add(value=conv, bias=biases)

# 把计算结果通过relu激活函数进行去线性化
actived_conv = tf.nn.relu(features=bias)

#actived_conv = tf.nn.sigmoid(features=bias)
#actived_conv = tf.nn.tanh(features=bias)

#### 2. 池化样例

In [4]:
# 池化有两种常用的池化
# tf.nn.max_pool最大值池化
# tf.nn.avg_pool均值池化

# ksize为池化层过滤器filter的尺寸，过滤器的头尾两个维度必须等于1
# 实际中使用最多的尺寸是[1,2,2,1]或者[1,3,3,1]
# strides为池化层过滤器步长
pool = tf.nn.max_pool(
    value=actived_conv, 
    ksize=[1,3,3,1], 
    strides=[1,2,2,1], 
    padding='SAME'
)


#### 理解strides参数
一般要求 strides的参数，strides[0] = strides[3] = 1

具体什么含义呢？

一般而言，对于输入张量（input tensor）有四维信息：[batch, height, width, channels]（分别表示 batch_size, 也即样本的数目，单个样本的行数和列数，样本的频道数，rgb图像就是三维的，灰度图像则是一维），对于一个二维卷积操作而言，其主要作用在 height, width上。

strides参数确定了滑动窗口在各个维度上移动的步数。一种常用的经典设置就是要求，strides[0]=strides[3]=1。

strides[0] = 1，也即在 batch 维度上的移动为 1，也就是不跳过任何一个样本，否则当初也不该把它们作为输入（input）
strides[3] = 1，也即在 channels 维度上的移动为 1，也就是不跳过任何一个颜色通道；

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(biases))
    print(sess.run(filter_weight))

[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1
  0.1]
[[[[  1.19266905e-01   1.25103787e-01   1.52913583e-02 ...,
     -3.90133709e-02   8.02899227e-02  -1.56643242e-01]
   [ -1.42708674e-01   7.30041191e-02  -1.04145315e-02 ...,
      5.10227270e-02  -5.01605682e-03   9.90026258e-03]
   [ -9.33057740e-02  -1.60699002e-02  -8.62591714e-02 ...,
      8.12473893e-02  -6.97776824e-02   1.09069169e-01]]

  [[  1.54431820e-01   7.03642610e-03   3.75612676e-02 ...,
     -8.86907279e-02   1.26131251e-01   1.83971718e-01]
   [ -4.30983938e-02  -1.38566837e-01  -8.62499401e-02 ...,
      4.78325114e-02   2.18250919e-02   1.85289886e-02]
   [  1.25046834e-01  -8.11936110e-02  -2.16353070e-02 ...,
      8.13222453e-02   6.54561818e-03  -8.46010670e-02]]

  [[  3.98844015e-03   2.84507368e-02  -9.40130204e-02 ...,
      1.77902579e-01  -1.90290555e-01   2.81488802e-03]
   [  1.07225798e-01  -5.85357957e-02  -5.95435016e-02 ...,
     -4.00721543e-02   1.17405854e-01  -1

#### LENET-5模型

```python
input: [width=32, height=32, color=1]

conv1:
filter: [width=5, height=5, input-dims=1, output-dims=6]
stride: [x-stride=1, y-stride=1]
output: [width=(32-5+1)/1=28, height=(32-5+1)/1=28, output-dims=6]

pool1:
filter: [width=2, height=2]
stride: [x-stride=2, y-stride=2]
output: [width=28/2=14, height=28/2=14, output-dims=6]

conv2:
filter: [width=5, height=5, input-dims=6, output-dims=16]
stride: [x-stride=1, y-stride=1]
output: [width=(14-5+1)/1=10, height=(14-5+1)/1=10, output-dims=16]

pool2:
filter: [width=2, height=2]
stride: [x-stride=2, y-stride=2]
output: [width=10/2=5, height=10/2=5, output-dims=16]

full-connect1:
w: [5*5*16=400, 120]
output: [120]

full-connect2:
w: [120, 84]
output: [84]

full-connect3:
w: [84, 10]
output: [10]
```